In [21]:
import json
from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm
import os


mode = 'val' # 'val' or 'train'
img_width = 1280
img_height = 720
root_folder = Path('BDDDataset')
images_folder = root_folder / 'images' / mode
labels_folder = root_folder / 'labels'
det_folder = root_folder / 'labels' / 'det' / mode
seg_folder = root_folder / 'labels' / 'seg' / mode


det_folder.mkdir(parents=True, exist_ok=True)
seg_folder.mkdir(parents=True, exist_ok=True)

detection_classes = {
    'pedestrian':   0,
    'rider':        1,
    'car':          2,
    'truck':        3,
    'bus':          4,
    'train':        5,
    'motorcycle':   6,
    'bicycle':      7,
    'traffic light': 8,
    'traffic sign': 9,
}

with open(labels_folder / f'{mode}.json', 'r') as f:
    data = json.load(f)

img_names = [img['name'] for img in data]
actual_imgs = images_folder.iterdir()
remove_imgs = [img for img in actual_imgs if img not in img_names]
for img in remove_imgs:
    os.remove(images_folder / img)

for img in tqdm(data):
    img_name = img['name']
    img_path = images_folder / img_name
    if not img_path.exists():
        continue

    with open(det_folder / (img_name.replace('.jpg', '.txt')), 'w') as f:
        for label in img['labels']:
            if label['category'] in detection_classes:
                # Convert pixel coordinates to YOLO format
                x_center = (label['box2d']['x1'] + label['box2d']['x2']) / (2 * img_width)
                y_center = (label['box2d']['y1'] + label['box2d']['y2']) / (2 * img_height)
                width = (label['box2d']['x2'] - label['box2d']['x1']) / img_width
                height = (label['box2d']['y2'] - label['box2d']['y1']) / img_height
                f.write(f"{detection_classes[label['category']]} {x_center:.4f} {y_center:.4f} {width:.4f} {height:.4f}\n")
        
    mask = np.zeros((img_height, img_width, 1), dtype=np.uint8)
    for label in img['labels']:
        if label['category'] == 'drivable area':
            poly = label['poly2d'][0]['vertices']
            if poly:
                poly = np.array([[(v[0], v[1]) for v in poly]], dtype=np.int32)
                cv2.fillPoly(mask, poly, 255)
    mask_path = seg_folder / (img_name)
    cv2.imwrite(str(mask_path), mask)

100%|██████████| 10000/10000 [00:03<00:00, 2616.23it/s]
